In [32]:
import pandas as pd
import numpy as np
import fastf1
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

2024-11-14 10:13:17.477964: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-14 10:13:17.624357: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-14 10:13:17.625378: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-14 10:13:18.742967: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [33]:
df = pd.read_csv('/home/riccardo/Documents/F1/dataset_plus_strait_lines_win.csv')
df = df.drop('Unnamed: 0',axis=1)

# Perceptron 64-32-1

In [ ]:
# Préparation des données
features = df[['X', 'Y', 'Angle', 'Straight_Line', 'Speed']].fillna(0)  # Remplir les NaN avec 0 ou une autre valeur
target = df['WIN']

In [34]:
# Normaliser les features
scaler = StandardScaler()
features = scaler.fit_transform(features)

In [35]:
# Diviser les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [36]:
# Construction du modèle
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Activation sigmoïde pour la classification binaire
])

In [37]:
# Compilation du modèle
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [38]:
# Entraînement du modèle
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
22/22 [==============================] - 1s 10ms/step - loss: -0.5911 - accuracy: 0.8082 - val_loss: -2.9736 - val_accuracy: 0.8693
Epoch 2/10
22/22 [==============================] - 0s 5ms/step - loss: -3.2657 - accuracy: 0.8679 - val_loss: -6.7358 - val_accuracy: 0.8693
Epoch 3/10
22/22 [==============================] - 0s 2ms/step - loss: -6.8220 - accuracy: 0.8679 - val_loss: -12.9261 - val_accuracy: 0.8693
Epoch 4/10
22/22 [==============================] - 0s 3ms/step - loss: -12.5796 - accuracy: 0.8679 - val_loss: -22.5141 - val_accuracy: 0.8693
Epoch 5/10
22/22 [==============================] - 0s 3ms/step - loss: -21.4413 - accuracy: 0.8679 - val_loss: -37.3625 - val_accuracy: 0.8693
Epoch 6/10
22/22 [==============================] - 0s 3ms/step - loss: -34.7445 - accuracy: 0.8679 - val_loss: -59.0508 - val_accuracy: 0.8693
Epoch 7/10
22/22 [==============================] - 0s 3ms/step - loss: -52.8681 - accuracy: 0.8679 - val_loss: -90.1402 - val_accuracy: 0.8

In [ ]:
# Évaluation du modèle
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy * 100:.2f}%")

7/7 [==============================] - 0s 2ms/step - loss: -260.7820 - accuracy: 0.8455
Accuracy: 84.55%


# model 2 